In [ ]:
import requests
import pandas as pd

api_url = "https://ows.goszakup.gov.kz/v3/refs/ref_abp"
token = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}
file = "admin_budget_program.csv"

def retrieve_admin_budget_program():
    """Получает администраторов бюджетных программ через API и сохраняет в CSV."""
    admin_list = []
    next_page = None

    while True:
        try:
            params = {"limit": 500}
            if next_page:
                params["page"] = "next"
                params["search_after"] = next_page

            response = requests.get(api_url, headers=headers, params=params, timeout=10)
            response.raise_for_status()
            response_data = response.json()

            records = response_data.get("items", [])
            if not records:
                print("Все данные загружены.")
                break

            for record in records:
                admin_list.append({
                    "id": record.get("id"),
                    "shortname_kz": record.get("shortname_kz"),
                    "shortname_ru": record.get("shortname_ru"),
                    "name_kz": record.get("name_kz"),
                    "name_ru": record.get("name_ru"),
                    "code": record.get("code")
                })

            print(f"Загружено записей: {len(admin_list)}")
            next_page = response_data.get("next_page")
            if not next_page:
                break

        except requests.exceptions.RequestException as err:
            print(f"Ошибка запроса: {err}")
            break

    export_to_csv(admin_list)

def export_to_csv(dataset):
    """Записывает данные в CSV-файл."""
    if not dataset:
        print("Нет данных для экспорта.")
        return

    data_frame = pd.DataFrame(dataset)
    data_frame.to_csv(
        file,
        index=False,
        encoding="utf-8-sig",
        sep="|",
        quotechar="'",
        escapechar="\\"
    )
    print(f"Данные сохранены в {file}")

retrieve_admin_budget_program()